# Importing Libraries

In [1]:
import os
import shutil
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

# Desing Basic Features

We shall now design the following **Features**:

1. question1_freq = Frequency of question1

2. question2_freq = Frequency of question2

3. quesiton1_len = Length of question1

4. quesiton2_len = Length of question2

5. question1_n_words = Number of words in question1

6. question2_n_words = Number of words in question2

7. len_diff = absoulte value of question1 length - question2 length

8. words_diff = absoulte value of question1 number of words - question2 number of words.

8. words_common = Number of common unique words in Question 1 and Question 2

9. words_share = word_common / (len(quesiton1) + len(question2))

10. frequency_sum = frequency of quesiton1 + frequency of quesiton2

10. frequency_diff = absolute value of frequency of quesiton1 - frequency of quesiton2

We will now design the feature for training data only, later on after we finish modeling based on our training data, we are gonna build similar features in term of test data as well.

In [ ]:
train_data = pd.read_csv("/Data/train_data.csv")

In [ ]:
train_data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,84246,167321,167322,Which team will win the 2018 World Cup?,What teams will be in the 2018 World Cup?,1
1,93023,184683,184684,"Which city is better to live, Dubai or Riyadh?",Is Riyadh a safe city to work?,0
2,1134,2269,2270,What were Hillary Clinton's positive achieveme...,What is Hillary Clinton's greatest achievement...,1
3,112175,222414,222415,Where can I find a list of companies in india ...,There are few startups in the US whose job des...,0
4,277626,545847,545848,What video will show about 10 different extras...,Will the fight between the U.S. & Russia end l...,0


In [ ]:
train_data["question1_len"] = train_data["question1"].str.len()
train_data["question2_len"] = train_data["question2"].str.len()

In [ ]:
train_data.head()

,id,qid1,qid2,question1,question2,is_duplicate,question1_len,question2_len
0,84246,167321,167322,Which team will win the 2018 World Cup?,What teams will be in the 2018 World Cup?,1,39,41
1,93023,184683,184684,"Which city is better to live, Dubai or Riyadh?",Is Riyadh a safe city to work?,0,46,30
2,1134,2269,2270,What were Hillary Clinton's positive achieveme...,What is Hillary Clinton's greatest achievement...,1,72,71
3,112175,222414,222415,Where can I find a list of companies in india ...,There are few startups in the US whose job des...,0,72,148
4,277626,545847,545848,What video will show about 10 different extras...,Will the fight between the U.S. & Russia end l...,0,153,83


In [ ]:
q1_words, q2_words = [], []

for i, j in zip(train_data.question1.values, train_data.question2.values):
  q1_words.append(len(i.split(" ")))
  q2_words.append(len(j.split(" ")))

train_data["question1_n_words"] = q1_words
train_data["question2_n_words"] = q2_words

In [ ]:
train_data.head()

,id,qid1,qid2,question1,question2,is_duplicate,question1_len,question2_len,question1_n_words,question2_n_words
0,84246,167321,167322,Which team will win the 2018 World Cup?,What teams will be in the 2018 World Cup?,1,39,41,8,9
1,93023,184683,184684,"Which city is better to live, Dubai or Riyadh?",Is Riyadh a safe city to work?,0,46,30,9,7
2,1134,2269,2270,What were Hillary Clinton's positive achieveme...,What is Hillary Clinton's greatest achievement...,1,72,71,10,11
3,112175,222414,222415,Where can I find a list of companies in india ...,There are few startups in the US whose job des...,0,72,148,15,26
4,277626,545847,545848,What video will show about 10 different extras...,Will the fight between the U.S. & Russia end l...,0,153,83,26,17


In [ ]:
train_data["question1_freq"] = train_data.groupby("qid1")["qid1"].transform("count")
train_data["question2_freq"] = train_data.groupby("qid2")["qid2"].transform("count")

In [ ]:
train_data.head()

,id,qid1,qid2,question1,question2,is_duplicate,question1_len,question2_len,question1_n_words,question2_n_words,question1_freq,question2_freq
0,84246,167321,167322,Which team will win the 2018 World Cup?,What teams will be in the 2018 World Cup?,1,39,41,8,9,1,1
1,93023,184683,184684,"Which city is better to live, Dubai or Riyadh?",Is Riyadh a safe city to work?,0,46,30,9,7,1,1
2,1134,2269,2270,What were Hillary Clinton's positive achieveme...,What is Hillary Clinton's greatest achievement...,1,72,71,10,11,1,1
3,112175,222414,222415,Where can I find a list of companies in india ...,There are few startups in the US whose job des...,0,72,148,15,26,1,1
4,277626,545847,545848,What video will show about 10 different extras...,Will the fight between the U.S. & Russia end l...,0,153,83,26,17,1,1


In [ ]:
train_data[train_data["question1_freq"] > 20]

,id,qid1,qid2,question1,question2,is_duplicate,question1_len,question2_len,question1_n_words,question2_n_words,question1_freq,question2_freq
65576,187946,7863,371102,"What is the cheapest, painless, easiest way to...",What is the easiest way of committing suicide?,1,62,46,10,8,25,1
97392,364121,7863,712782,"What is the cheapest, painless, easiest way to...",What is the most painless way to commit suicide?,1,62,48,10,9,25,1
107389,168308,7863,332624,"What is the cheapest, painless, easiest way to...",What is the most effective way to suicide?,1,62,42,10,8,25,1
113129,130973,7863,259413,"What is the cheapest, painless, easiest way to...",What is the best way to die?,0,62,28,10,7,25,1
126783,49818,7863,99168,"What is the cheapest, painless, easiest way to...",What's the absolute easiest way to commit suic...,1,62,50,10,8,25,1
135347,151721,7863,300120,"What is the cheapest, painless, easiest way to...",What is the least painful and best way to comm...,1,62,57,10,11,25,1
148306,214462,7863,422917,"What is the cheapest, painless, easiest way to...",How do I give up the thought of committing sui...,0,62,51,10,10,25,1
153181,272143,7863,535199,"What is the cheapest, painless, easiest way to...",What is an easy way to commit suicide?,1,62,38,10,8,25,1
154645,118701,7863,235261,"What is the cheapest, painless, easiest way to...",What can be the most insane ways to commit sui...,1,62,51,10,10,25,1
160967,50027,7863,99580,"What is the cheapest, painless, easiest way to...",What is the cheapest method to commit suicide?,1,62,46,10,8,25,1


In [ ]:
train_data[train_data["question2_freq"] > 30]

,id,qid1,qid2,question1,question2,is_duplicate,question1_len,question2_len,question1_n_words,question2_n_words,question1_freq,question2_freq,len_diff
1685,94193,186993,65333,"How do i reduce fat around chest, thighs and h...",What are the best ways to reduce and eliminate...,0,58,79,11,14,1,38,21
13776,225426,444298,65333,How can I gain muscle to lose body fat?,What are the best ways to reduce and eliminate...,0,39,79,9,14,1,38,40
18551,303540,596052,65333,What are some of the best bodyweight exercises...,What are the best ways to reduce and eliminate...,0,68,79,12,14,1,38,11
30624,246372,485180,65333,How can we reduce fatty thighs?,What are the best ways to reduce and eliminate...,0,31,79,6,14,1,38,48
53343,169796,335537,65333,Can reflexology remove belly fat?,What are the best ways to reduce and eliminate...,0,33,79,5,14,1,38,46
60235,115416,228792,65333,What is the best way to flat belly?,What are the best ways to reduce and eliminate...,0,35,79,8,14,1,38,44
65153,186182,367646,65333,How do I get rid of fat?,What are the best ways to reduce and eliminate...,0,24,79,7,14,1,38,55
67165,124833,247340,65333,How can I have a flat belly? What do you have ...,What are the best ways to reduce and eliminate...,0,86,79,21,14,1,38,7
75882,231220,455609,65333,What's the way to lose belly fat?,What are the best ways to reduce and eliminate...,0,33,79,7,14,1,38,46
78993,330727,648558,65333,Is planking exercise dangerous?,What are the best ways to reduce and eliminate...,0,31,79,4,14,1,38,48


In [ ]:
train_data["len_diff"] = abs(train_data["question1_len"] - train_data["question2_len"])

In [ ]:
train_data.head()

,id,qid1,qid2,question1,question2,is_duplicate,question1_len,question2_len,question1_n_words,question2_n_words,question1_freq,question2_freq,len_diff
0,84246,167321,167322,Which team will win the 2018 World Cup?,What teams will be in the 2018 World Cup?,1,39,41,8,9,1,1,2
1,93023,184683,184684,"Which city is better to live, Dubai or Riyadh?",Is Riyadh a safe city to work?,0,46,30,9,7,1,1,16
2,1134,2269,2270,What were Hillary Clinton's positive achieveme...,What is Hillary Clinton's greatest achievement...,1,72,71,10,11,1,1,1
3,112175,222414,222415,Where can I find a list of companies in india ...,There are few startups in the US whose job des...,0,72,148,15,26,1,1,76
4,277626,545847,545848,What video will show about 10 different extras...,Will the fight between the U.S. & Russia end l...,0,153,83,26,17,1,1,70


In [ ]:
train_data["words_diff"] = abs(train_data["question1_n_words"] - train_data["question2_n_words"])

In [ ]:
train_data.head()

,id,qid1,qid2,question1,question2,is_duplicate,question1_len,question2_len,question1_freq,question2_freq,len_diff,question1_n_words,question2_n_words,words_diff
0,84246,167321,167322,Which team will win the 2018 World Cup?,What teams will be in the 2018 World Cup?,1,39,41,1,1,2,8,9,1
1,93023,184683,184684,"Which city is better to live, Dubai or Riyadh?",Is Riyadh a safe city to work?,0,46,30,1,1,16,9,7,2
2,1134,2269,2270,What were Hillary Clinton's positive achieveme...,What is Hillary Clinton's greatest achievement...,1,72,71,1,1,1,10,11,1
3,112175,222414,222415,Where can I find a list of companies in india ...,There are few startups in the US whose job des...,0,72,148,1,1,76,15,26,11
4,277626,545847,545848,What video will show about 10 different extras...,Will the fight between the U.S. & Russia end l...,0,153,83,1,1,70,26,17,9


In [ ]:
common_words = []

for i, j in zip(train_data["question1"].values, train_data["question2"].values):

  s1 = set([word.lower() for word in i.split()])
  s2 = set([word.lower() for word in j.split()])

  common_words.append(len(s1 & s2))

train_data["words_common"] = common_words

In [ ]:
train_data.head()

,id,qid1,qid2,question1,question2,is_duplicate,question1_len,question2_len,question1_freq,question2_freq,len_diff,question1_n_words,question2_n_words,words_diff,words_common
0,84246,167321,167322,Which team will win the 2018 World Cup?,What teams will be in the 2018 World Cup?,1,39,41,1,1,2,8,9,1,5
1,93023,184683,184684,"Which city is better to live, Dubai or Riyadh?",Is Riyadh a safe city to work?,0,46,30,1,1,16,9,7,2,3
2,1134,2269,2270,What were Hillary Clinton's positive achieveme...,What is Hillary Clinton's greatest achievement...,1,72,71,1,1,1,10,11,1,7
3,112175,222414,222415,Where can I find a list of companies in india ...,There are few startups in the US whose job des...,0,72,148,1,1,76,15,26,11,4
4,277626,545847,545848,What video will show about 10 different extras...,Will the fight between the U.S. & Russia end l...,0,153,83,1,1,70,26,17,9,6


In [ ]:
train_data["words_share"] = train_data["words_common"] / (train_data["question1_n_words"] + train_data["question2_n_words"])

In [ ]:
train_data.head()

,id,qid1,qid2,question1,question2,is_duplicate,question1_len,question2_len,question1_freq,question2_freq,len_diff,question1_n_words,question2_n_words,words_diff,words_common,words_share
0,84246,167321,167322,Which team will win the 2018 World Cup?,What teams will be in the 2018 World Cup?,1,39,41,1,1,2,8,9,1,5,0.294118
1,93023,184683,184684,"Which city is better to live, Dubai or Riyadh?",Is Riyadh a safe city to work?,0,46,30,1,1,16,9,7,2,3,0.187500
2,1134,2269,2270,What were Hillary Clinton's positive achieveme...,What is Hillary Clinton's greatest achievement...,1,72,71,1,1,1,10,11,1,7,0.333333
3,112175,222414,222415,Where can I find a list of companies in india ...,There are few startups in the US whose job des...,0,72,148,1,1,76,15,26,11,4,0.097561
4,277626,545847,545848,What video will show about 10 different extras...,Will the fight between the U.S. & Russia end l...,0,153,83,1,1,70,26,17,9,6,0.139535


In [ ]:
train_data["frequency_sum"] = train_data["question1_freq"] + train_data["question2_freq"]
train_data["frequency_diff"] = abs(train_data["question1_freq"] - train_data["question2_freq"])

In [ ]:
train_data.head()

,id,qid1,qid2,question1,question2,is_duplicate,question1_len,question2_len,question1_freq,question2_freq,len_diff,question1_n_words,question2_n_words,words_diff,words_common,words_share,frequency_sum,frequency_diff
0,84246,167321,167322,Which team will win the 2018 World Cup?,What teams will be in the 2018 World Cup?,1,39,41,1,1,2,8,9,1,5,0.294118,2,0
1,93023,184683,184684,"Which city is better to live, Dubai or Riyadh?",Is Riyadh a safe city to work?,0,46,30,1,1,16,9,7,2,3,0.187500,2,0
2,1134,2269,2270,What were Hillary Clinton's positive achieveme...,What is Hillary Clinton's greatest achievement...,1,72,71,1,1,1,10,11,1,7,0.333333,2,0
3,112175,222414,222415,Where can I find a list of companies in india ...,There are few startups in the US whose job des...,0,72,148,1,1,76,15,26,11,4,0.097561,2,0
4,277626,545847,545848,What video will show about 10 different extras...,Will the fight between the U.S. & Russia end l...,0,153,83,1,1,70,26,17,9,6,0.139535,2,0


In [ ]:
test_data = pd.read_csv("/Data/test_data.csv")

In [ ]:
test_data["question1_len"] = test_data["question1"].str.len()
test_data["question2_len"] = test_data["question2"].str.len()

We can cetrainly put this part of code as a function so we don't have to write it two times but i just do this for clarification.

In [ ]:
q1_words, q2_words = [], []

for i, j in zip(test_data.question1.values, test_data.question2.values):
  q1_words.append(len(i.split(" ")))
  q2_words.append(len(j.split(" ")))

test_data["question1_n_words"] = q1_words
test_data["question2_n_words"] = q2_words

In [ ]:
test_data["question1_freq"] = test_data.groupby("qid1")["qid1"].transform("count")
test_data["question2_freq"] = test_data.groupby("qid2")["qid2"].transform("count")

In [ ]:
test_data["len_diff"] = abs(test_data["question1_len"] - test_data["question2_len"])

In [ ]:
test_data["words_diff"] = abs(test_data["question1_n_words"] - test_data["question2_n_words"])

In [ ]:
common_words = []

for i, j in zip(test_data["question1"].values, test_data["question2"].values):

  s1 = set([word.lower() for word in i.split()])
  s2 = set([word.lower() for word in j.split()])

  common_words.append(len(s1 & s2))

test_data["words_common"] = common_words

In [ ]:
test_data["words_share"] = test_data["words_common"] / (test_data["question1_n_words"] + test_data["question2_n_words"])

In [ ]:
test_data["frequency_sum"] = test_data["question1_freq"] + test_data["question2_freq"]
test_data["frequency_diff"] = abs(test_data["question1_freq"] - test_data["question2_freq"])

In [ ]:
print("Number of Features in train data =", train_data.shape[1])
print("Number of Features in test data =", test_data.shape[1])

Number of Features in train data = 18
Number of Features in test data = 18


In [ ]:
print("The Features we have designed so far is:")
train_data.columns.tolist()[6:]

The Features we have designed so far is:


['question1_len',
 'question2_len',
 'question1_freq',
 'question2_freq',
 'len_diff',
 'question1_n_words',
 'question2_n_words',
 'words_diff',
 'words_common',
 'words_share',
 'frequency_sum',
 'frequency_diff']

From now on we will use compression while exporting our data into csv or even pickle files, this will massively reduce size of features data on disk but it will load a little slower.

In [ ]:
train_data.to_csv("/Data/train_data.csv", index = False, compression = "bz2")
test_data.to_csv("/Data/test_data.csv", index = False, compression = "bz2")